In [1]:
import os
os.environ["R_HOME"] = "/home/huangzhuoli/mambaforge/envs/tensorqtl/lib/R"
os.environ["R_LIBS_USER"] = "/home/huangzhuoli/mambaforge/envs/tensorqtl/lib/R/library"
import pandas as pd
import torch
import tensorqtl

In [2]:
import numpy as np

In [ ]:
pr = tensorqtl.genotypeio.PlinkReader('/CIMA/genetics/qc/10.maf01')
# load genotypes and variants into data frames
genotype_df = pr.load_genotypes()
variant_df = pr.bim.set_index('snp')[['chrom', 'pos']]

Mapping files: 100%|██████████| 3/3 [41:46<00:00, 835.50s/it]


In [ ]:
genotype_df.to_parquet('/CIMA/Data/413_sample_genotype.parquet')

In [ ]:
variant_df.to_parquet('/CIMA/Data/413_sample_variant_df.parquet')

In [ ]:
#解决R语言把“-”换成“."的问题
tss = pd.read_csv('/CIMA/Make_Gene_TSS/tss.bed',sep = ' ')
#为了和R得到的结果相符合
tss['gene_id_map'] = tss['gene_id'].str.replace('-', '.', regex=False)
# 创建字典，映射 gene_id_map 到 gene_id，只包含 B 数据框中出现的列
gene_map_dict = pd.Series(tss.gene_id.values, index=tss.gene_id_map).to_dict()

In [ ]:
celltype_over_70_eQTL = pd.read_csv('/CIMA/Data/20241230_xQTL_cell_sample_number/20241230_celltype_70_scRNA.csv')
#tss = pd.read_csv('/CIMA/Make_Gene_TSS/tss.bed',sep = ' ')
#为了和R得到的结果相符合
#tss['gene_id'] = tss['gene_id'].str.replace('-', '.', regex=False)

for cell in celltype_over_70_eQTL['final_annotation']:
    print(f'processing_{cell}')
    #make_bed_file_eQTL
    ## Prepare phenotype data
    norm_expr = pd.read_csv(f'/CIMA/Data/eQTL/normal_dis/{cell}.csv', index_col=0)
    #解决R语言列名把“-”变成'.'的问题
    filtered_map_dict = {key: gene_map_dict[key] for key in norm_expr.columns}
    # 使用字典重命名 B 数据框的列
    norm_expr.rename(columns=filtered_map_dict, inplace=True)

    norm_expr = norm_expr.T
    norm_expr = norm_expr.reset_index()   # gene x sample

    pheno_file =  pd.merge(tss, norm_expr, right_on='index', left_on='gene_id', how='inner')
    pheno_file = pheno_file.drop(['index'], axis=1).rename(columns={'chr':'#chr'})
    pheno_file['#chr'] = pheno_file['#chr'].str.replace('chr','')    # required phenotype file

    if norm_expr.shape[0] != pheno_file.shape[0]:
        raise ValueError("The number of rows in the two dataframes are not equal.")
    else:
        pheno_file.to_csv(f'/CIMA/Data/eQTL/bed_file/{cell}.bed', sep='\t', index=False)

processing_Atypical_Bm_ITGAX
processing_Bn_IFIT3
processing_Bn_TCL1A
processing_Bn_IL6
processing_CD4_CTL_GZMH
processing_CD4_Tcm_CXCR5
processing_CD4_Tcm_IFI44L
processing_CD4_Tem_CCR7neg
processing_CD4_Tfh-like_CXCR5
processing_CD4_Th1-like_GZMK
processing_CD4_Th17-like_RORC
processing_CD4_Th22-like_CCR10
processing_CD4_Th_CCR4
processing_CD4_Th_CR1
processing_CD4_Th_LMNA
processing_CD4_Th_TNFRSF11A
processing_CD4_Tn_CCR7
processing_CD4_Tn_CXCR5
processing_CD4_Tn_LIMS1
processing_CD4_Tn_SOX4
processing_CD4_Tr1-like_IL10
processing_CD4_Treg_FCRL3
processing_CD4_Treg_FOXP3
processing_CD8_CTL_GZMB
processing_CD8_CTL_GZMK
processing_CD8_CTL_IFI44L
processing_CD8_Tcm_IFI44L
processing_CD8_Tem_CCR7neg
processing_CD8_Tn_CCR7
processing_CD8_Tn_SOX4
processing_Cycling_NK_MKI67
processing_Cycling_T_MKI67
processing_pre-T-like_CABP4
processing_HSPC_CD34
processing_ILC2_IL2RA
processing_MAIT_SLC4A10
processing_MK_GP9
processing_NKT_NCR1
processing_NKT_IFNG
processing_NK_bright_XCL1
processing_Pl

In [7]:
#处理caQTL
def calculate_midpoint(input_str):
    # 切分输入字符串，提取数字部分
    parts = input_str.split(':')[-1].split('-')
    
    # 提取数字calculate_midpoint
    start_num = int(parts[0])
    end_num = int(parts[1])
    
    # 计算中点
    midpoint = (start_num + end_num) // 2
    
    return midpoint

# 测试
input_str = "chr10:100006245-100006832"
midpoint = calculate_midpoint(input_str)
print("Midpoint:", midpoint)

vectorized_calculate_midpoint = np.vectorize(calculate_midpoint)

Midpoint: 100006538


In [ ]:
celltype_over_70_caQTL = pd.read_csv('/CIMA/Data/20241230_xQTL_cell_sample_number/20241230_celltype_70_scATAC.csv')

In [ ]:
for celltype in celltype_over_70_caQTL['final_annotation']:
    
    print(celltype)
    
    norm_expr = pd.read_csv(f'/CIMA/Data/caQTL/normal_dis/{celltype}.csv', index_col=0)
    norm_expr.columns = norm_expr.columns.str.replace('.', ':', 1).str.replace('.', '-', 1)
    norm_expr = norm_expr.T
    norm_expr = norm_expr.reset_index() 

    info = pd.DataFrame({'#chr':norm_expr['index'].str.split(":", expand=True)[0].str.replace('chr','')})
    info['start'] = vectorized_calculate_midpoint(norm_expr['index']) -1
    info['end'] = vectorized_calculate_midpoint(norm_expr['index'])
    info['gene_id'] = norm_expr['index']

    pheno_file =  pd.merge(info, norm_expr, right_on='index', left_on='gene_id', how='inner')
    pheno_file = pheno_file.drop(['index'], axis=1)
    pheno_file['#chr'] = pheno_file['#chr'].astype(int)
    pheno_file = pheno_file.sort_values(by = '#chr').reset_index(drop=True)
    pheno_file = pheno_file.groupby('#chr', sort=False, group_keys=False).apply(lambda x: x.sort_values('start')).reset_index(drop=True)
    
    if norm_expr.shape[0] != pheno_file.shape[0]:
        raise ValueError("The number of rows in the two dataframes are not equal.")
    else:
        pheno_file.to_csv(f'/CIMA/Data/caQTL/bed_file/{celltype}.bed', sep='\t', index=False)

Atypical_Bm_ITGAX


/tmp/ipykernel_1216842/3218016426.py:19: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  pheno_file = pheno_file.groupby('#chr', sort=False, group_keys=False).apply(lambda x: x.sort_values('start')).reset_index(drop=True)


Bn_TCL1A


/tmp/ipykernel_1216842/3218016426.py:19: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  pheno_file = pheno_file.groupby('#chr', sort=False, group_keys=False).apply(lambda x: x.sort_values('start')).reset_index(drop=True)


CD4_CTL_GZMH


...

/tmp/ipykernel_1216842/3218016426.py:19: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  pheno_file = pheno_file.groupby('#chr', sort=False, group_keys=False).apply(lambda x: x.sort_values('start')).reset_index(drop=True)


pDC_IRF4


/tmp/ipykernel_1216842/3218016426.py:19: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  pheno_file = pheno_file.groupby('#chr', sort=False, group_keys=False).apply(lambda x: x.sort_values('start')).reset_index(drop=True)


pre-Switched_Bm_JAM3


/tmp/ipykernel_1216842/3218016426.py:19: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  pheno_file = pheno_file.groupby('#chr', sort=False, group_keys=False).apply(lambda x: x.sort_values('start')).reset_index(drop=True)
